# Домашняя работа 

В этой работы мы будем учиться предсказывать зарплату data scientist-ов в зависимочти от ряда факторов с помощью градиентоного бустинга

Цель домашней работы научиться предсказывать зарплаты (salary_in_usd) по ряды факторов 

work_year: The number of years of work experience in the field of data science.

experience_level: The level of experience, such as Junior, Senior, or Lead.

employment_type: The type of employment, such as Full-time or Contract.

job_title: The specific job title or role, such as Data Analyst or Data Scientist.

salary: The salary amount for the given job.

salary_currency: The currency in which the salary is denoted.

salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

employee_residence: The country or region where the employee resides.

remote_ratio: The percentage of remote work offered in the job.

company_location: The location of the company or organization.

company_size: The company’s size is categorized as Small, Medium, or Large.

In [3]:
#!pip install xgboost

     ---------------------------------------- 70.9/70.9 MB 5.3 MB/s eta 0:00:00


In [190]:
#!pip install lightgbm

     ---------------------------------------- 1.0/1.0 MB 5.0 MB/s eta 0:00:00


In [214]:
!pip install catboost

In [38]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("ds_salaries.csv")

In [3]:
df.head()

,Unnamed: 0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


## Задание 1 (0.5 балла)

Разделите выборку на train, val, test (80%, 10%, 10%) в качестве таргета выберите salary_in_usd, удалите из признаков (salary) чтобы избежать лика в данных.

In [108]:
data = df
data.drop(columns = 'salary', inplace = True)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
data['years'] = 2023 - data['work_year']
X=data[['years', 'remote_ratio']]
y = data['salary_in_usd']
X_train, X_test1, y_train, y_test1 = train_test_split(X,y, train_size=0.8, random_state = 123)
X_valid, X_test, y_valid, y_test = train_test_split(X_test1,y_test1, train_size=0.5, random_state = 123)

In [6]:
help(mean_absolute_percentage_error)

Help on function mean_absolute_percentage_error in module sklearn.metrics._regression:

mean_absolute_percentage_error(y_true, y_pred, *, sample_weight=None, multioutput='uniform_average')
    Mean absolute percentage error (MAPE) regression loss.
    
    Note here that the output is not a percentage in the range [0, 100]
    and a value of 100 does not mean 100% but 1e2. Furthermore, the output
    can be arbitrarily high when `y_true` is small (which is specific to the
    metric) or when `abs(y_true - y_pred)` is large (which is common for most
    regression metrics). Read more in the
    :ref:`User Guide <mean_absolute_percentage_error>`.
    
    .. versionadded:: 0.24
    
    Parameters
    ----------
    y_true : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Ground truth (correct) target values.
    
    y_pred : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Estimated target values.
    
    sample_weight : array-like of shape (n_samp

## Задание 2 (0.5 балла)

Обучите модель линейной регресии и оцените её качество через mape и rmse

In [7]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
pred_val = lr.predict(X_valid)
pred_test = lr.predict(X_test)

In [8]:
print('MAPE validation:', mean_absolute_percentage_error(y_valid, pred_val))
print('MAPE test:', mean_absolute_percentage_error(y_test, pred_test))
print('RMSE validation:', mean_squared_error(y_valid, pred_val, squared=False))
print('RMSE test:', mean_squared_error(y_test, pred_test, squared=False))

MAPE validation: 1.377095337420564
MAPE test: 0.8131311592903949
RMSE validation: 59451.861167384915
RMSE test: 83099.6381739832


## Задание 3 (0.5 балла)

Обучите модель бустинга выберите любую из трех библиотек catboost, xgboost, lightgbm и оцените её качество через mape и rmse

In [9]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

xg = XGBRegressor(learning_rate=0.1,
    n_estimators=150,
    eval_metric=mean_squared_error
)

xg.fit(X_train, y_train)
pred_val = xg.predict(X_valid)
pred_test = xg.predict(X_test)

print('MAPE validation:', mean_absolute_percentage_error(y_valid, pred_val))
print('MAPE test:', mean_absolute_percentage_error(y_test, pred_test))
print('RMSE validation:', mean_squared_error(y_valid, pred_val, squared=False))
print('RMSE test:', mean_squared_error(y_test, pred_test, squared=False))

MAPE validation: 1.380751964925216
MAPE test: 0.794523584919646
RMSE validation: 59328.97684268591
RMSE test: 81980.21717183548


## Задание 4 (0.5 балла)

Далее предобработайте категориальные признаки с помощью onehot кодирования, labelencoder и счетчиков (mean target encoding) сравните качество моделей с помощью mape и rmse. Необходимо обучить две модели линейную и бустинг. Какие выводы можно сделать? Выберите лучшую из моделей и дальше будем работать с ней

In [10]:
data['experience_level'].unique()
#, 'employment_type', 'job_title']

array(['MI', 'SE', 'EN', 'EX'], dtype=object)

In [11]:
data['employment_type'].unique()

array(['FT', 'CT', 'PT', 'FL'], dtype=object)

In [12]:
data.dtypes

Unnamed: 0             int64
work_year              int64
experience_level      object
employment_type       object
job_title             object
salary                 int64
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
years                  int64
dtype: object

In [109]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
X = pd.get_dummies(data.drop(columns = ['salary_in_usd']), columns=['experience_level', 'employment_type', 'job_title', 'employee_residence', 'company_location',  'salary_currency', 'work_year',
                                     'company_size'], drop_first= True)
X_train, X_test1, y_train, y_test1 = train_test_split(X,y, train_size=0.8, random_state = 123)
X_valid, X_test, y_valid, y_test = train_test_split(X_test1,y_test1, train_size=0.5, random_state = 123)

In [110]:
lr = LinearRegression()
xg = XGBRegressor(learning_rate=0.1,
    n_estimators=150,
    eval_metric=mean_squared_error
)
lr.fit(X_train, y_train)
xg.fit (X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False,
             eval_metric=<function mean_squared_error at 0x0000013529DAD900>,
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=150, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None, ...)

In [111]:
pred_val = lr.predict(X_valid)
pred_test = lr.predict(X_test)
print('lr MAPE validation:', mean_absolute_percentage_error(y_valid, pred_val))
print('lr MAPE test:', mean_absolute_percentage_error(y_test, pred_test))
print('lr RMSE validation:', mean_squared_error(y_valid, pred_val, squared=False))
print('lr RMSE test:', mean_squared_error(y_test, pred_test, squared=False))

lr MAPE validation: 1.03139863889167
lr MAPE test: 0.49463012002024986
lr RMSE validation: 73425.1411029403
lr RMSE test: 62805.99812922009


In [112]:
pred_val = xg.predict(X_valid)
pred_test = xg.predict(X_test)

print('MAPE validation:', mean_absolute_percentage_error(y_valid, pred_val))
print('MAPE test:', mean_absolute_percentage_error(y_test, pred_test))
print('RMSE validation:', mean_squared_error(y_valid, pred_val, squared=False))
print('RMSE test:', mean_squared_error(y_test, pred_test, squared=False))

MAPE validation: 0.70986000374719
MAPE test: 0.44615015552006587
RMSE validation: 53639.59477561858
RMSE test: 63830.1708948559


In [113]:
X = data.drop(columns = 'salary_in_usd')
for i in ['experience_level', 'employment_type', 'job_title', 'employee_residence', 'company_location',  'salary_currency', 'work_year',
                                     'company_size']:
    le = LabelEncoder()
    le.fit(X[i])
    data[i] = le.transform(data[i])
X_train, X_test1, y_train, y_test1 = train_test_split(X,y, train_size=0.8, random_state = 123)
X_valid, X_test, y_valid, y_test = train_test_split(X_test1,y_test1, train_size=0.5, random_state = 123)

In [114]:
lr = LinearRegression()
xg = XGBRegressor(learning_rate=0.1,
    n_estimators=150,
    eval_metric=mean_squared_error
)
lr.fit(X_train, y_train)
xg.fit (X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False,
             eval_metric=<function mean_squared_error at 0x0000013529DAD900>,
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=150, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None, ...)

In [115]:
pred_val = lr.predict(X_valid)
pred_test = lr.predict(X_test)
print('lr MAPE validation:', mean_absolute_percentage_error(y_valid, pred_val))
print('lr MAPE test:', mean_absolute_percentage_error(y_test, pred_test))
print('lr RMSE validation:', mean_squared_error(y_valid, pred_val, squared=False))
print('lr RMSE test:', mean_squared_error(y_test, pred_test, squared=False))

lr MAPE validation: 1.1264377322740124
lr MAPE test: 0.5488870685610068
lr RMSE validation: 51087.3251155027
lr RMSE test: 68906.88503153445


In [116]:
pred_val = xg.predict(X_valid)
pred_test = xg.predict(X_test)

print('MAPE validation:', mean_absolute_percentage_error(y_valid, pred_val))
print('MAPE test:', mean_absolute_percentage_error(y_test, pred_test))
print('RMSE validation:', mean_squared_error(y_valid, pred_val, squared=False))
print('RMSE test:', mean_squared_error(y_test, pred_test, squared=False))

MAPE validation: 0.7533504270334176
MAPE test: 0.3958105435523222
RMSE validation: 52853.0075823142
RMSE test: 69195.05418328871


In [117]:
#выводы: лучшая - бустинг с onehot, тк RMSE не сильно отличается от линрега, но эта моделька более оптимальна по MAPE

## Задание 5 (0.5 балла)

Покажите, где ошибается ваша модель. Выведите топ 20 примеров с наибольшей ошибкой. Проанализируете их, постарайтесь выделить группы с причинами ошибок в моделе. Какие выводы можно сделать? Что стоит изменить в данных чтобы улучшить качество модели?

In [187]:
X_test.columns

Index(['job_title', 'company_location', 'company_size', 'years',
       'experience_level_0', 'experience_level_1', 'experience_level_2',
       'experience_level_3', 'employee_residence_0', 'employee_residence_1',
       'employee_residence_2', 'employee_residence_3', 'employee_residence_4',
       'employee_residence_5', 'employee_residence_6', 'employee_residence_7',
       'employee_residence_8', 'employee_residence_9', 'employee_residence_10',
       'employee_residence_11', 'employee_residence_12',
       'employee_residence_13', 'employee_residence_14',
       'employee_residence_15', 'employee_residence_16',
       'employee_residence_17', 'employee_residence_18',
       'employee_residence_19', 'employee_residence_20',
       'employee_residence_21', 'employee_residence_22',
       'employee_residence_23', 'employee_residence_24',
       'employee_residence_25', 'employee_residence_26',
       'employee_residence_27', 'employee_residence_28',
       'employee_residence_29', 'e

In [188]:
y_pred = xg.predict(X_test)
sort_target = np.array(X_test)[np.argsort(y_pred)[:20]]
pd.DataFrame(data=sort_target[0:,1:], index=sort_target[0:,0], columns=sort_target[0,1:])

,39,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
13,39,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
10,31,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
38,28,0,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
22,17,1,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
17,18,2,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
17,40,0,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
17,18,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
22,16,0,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12,48,1,2,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
12,16,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [189]:
sorted((y_pred-y_test))

[-304471.4375,
 -116898.65625,
 -59828.40625,
 -58546.625,
 -49901.484375,
 -35240.25,
 -33949.625,
 -24249.625,
 -22404.703125,
 -20798.375,
 -18807.3984375,
 -17787.99609375,
 -16458.7265625,
 -15449.625,
 -12849.625,
 -9175.984375,
 -5866.0615234375,
 -5660.578125,
 -5455.70703125,
 -5100.578125,
 -3557.03125,
 -2580.05859375,
 -1207.4140625,
 -596.203125,
 546.1015625,
 872.5859375,
 1743.375,
 6453.375,
 7201.03125,
 8754.6484375,
 11453.375,
 12343.93359375,
 14345.5,
 17831.38671875,
 20905.6875,
 21954.82421875,
 23097.3515625,
 24342.3515625,
 27372.90625,
 27512.953125,
 27624.98046875,
 28250.40625,
 29345.8671875,
 30044.90625,
 30339.421875,
 33633.9921875,
 35237.42578125,
 39580.375,
 41615.9375,
 44519.890625,
 45208.21875,
 47201.03125,
 47339.421875,
 54819.625,
 63518.8125,
 65105.578125,
 68137.96875,
 85455.578125,
 87208.21875,
 124171.375,
 144795.609375]

In [60]:
#выводы: скорее всего, модель переобучилась :( (нижниие веса, ЭТО ЧТОООООО)). Будем пробовать убирать шум + по-разному кодировать категориальные признаки

## Задание 6 (0.5 балла)

Придумайте признаки для улучшения качества модели на основе предыдущего пункта. Как вам кажется какими признаками можно улучшить качетсво модели? Реализуйте признаки и проверьте улучшилось ли качество модели. Полный бал за задание ставится даже при отсутствии улучшения качества модели, важно попробовать проверить свои гипотезы и сделать выводы

In [138]:
from sklearn.metrics import mean_absolute_error

In [200]:
#почистим датасет от шумовых признаков (Id, валюты и прочего). Также вряд ли на зп влияет employment type.
X = data.drop(columns = ['salary_in_usd', 'Unnamed: 0',  'salary_currency', 'work_year', 'employment_type'])
X = pd.get_dummies(X, columns = ['experience_level', 'employee_residence',
                                    'remote_ratio'])

#Можно оставить label кодировку company size, job title и company_location

X_train, X_test1, y_train, y_test1 = train_test_split(X,y, train_size=0.8, random_state = 123)
X_valid, X_test, y_valid, y_test = train_test_split(X_test1,y_test1, train_size=0.5, random_state = 123)


xg = XGBRegressor(learning_rate=0.1,
    n_estimators=600,
    eval_metric=mean_absolute_error
)
xg.fit (X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False,
             eval_metric=<function mean_absolute_error at 0x0000013529DAD750>,
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=600, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None, ...)

In [178]:
pred_val = xg.predict(X_valid)
pred_test = xg.predict(X_test)

print('MAPE validation:', mean_absolute_percentage_error(y_valid, pred_val))
print('MAPE test:', mean_absolute_percentage_error(y_test, pred_test))
print('RMSE validation:', mean_squared_error(y_valid, pred_val, squared=False))
print('RMSE test:', mean_squared_error(y_test, pred_test, squared=False))

MAPE validation: 0.7571727099118783
MAPE test: 0.4049865142464014
RMSE validation: 62160.68294780022
RMSE test: 58884.869140414005


In [229]:
#качество модели ЗАМЕТНО увеличилось просто при разном кодировании и стирании ненужных признаков. Я молодец, гипотезы подствердились

## Задание 7 (0.5 балла)

Теперь используете исключительно catboost. Обучить модель. Подберите оптимальные гиперпараметры. Используйте pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметры cat_features. Оцените качество итоговой модели, скорость обучения и скорость предсказания

In [228]:
from catboost import CatBoostRegressor
import time
cb = CatBoostRegressor(iterations=600, learning_rate = 0.01)
start = time.time()
cb.fit(X_train, y_train)
end = time.time()
pred_val = cb.predict(X_valid)
pred_test = cb.predict(X_test)
en = time.time()
print('MAPE validation:', mean_absolute_percentage_error(y_valid, pred_val))
print('MAPE test:', mean_absolute_percentage_error(y_test, pred_test))
print('RMSE validation:', mean_squared_error(y_valid, pred_val, squared=False))
print('RMSE test:', mean_squared_error(y_test, pred_test, squared=False))
print('Время обучения: {0} мс, Время предсказания: {1} мс'.format(end-start, en - end))

0:	learn: 69906.3109922	total: 2.01ms	remaining: 1.2s
1:	learn: 69658.4313978	total: 4.17ms	remaining: 1.25s
2:	learn: 69397.5365813	total: 5.16ms	remaining: 1.03s
3:	learn: 69126.5415594	total: 6.28ms	remaining: 936ms
4:	learn: 68898.7562282	total: 7.52ms	remaining: 895ms
5:	learn: 68648.6680628	total: 8.63ms	remaining: 855ms
6:	learn: 68419.2175069	total: 9.66ms	remaining: 818ms
7:	learn: 68190.4315552	total: 10.6ms	remaining: 787ms
8:	learn: 67936.8138751	total: 11.8ms	remaining: 772ms
9:	learn: 67689.4852254	total: 12.8ms	remaining: 754ms
10:	learn: 67441.1602392	total: 13.8ms	remaining: 739ms
11:	learn: 67223.3148880	total: 14.8ms	remaining: 724ms
12:	learn: 67020.4342348	total: 15.6ms	remaining: 705ms
13:	learn: 66775.8375405	total: 16.6ms	remaining: 696ms
14:	learn: 66542.3865068	total: 17.6ms	remaining: 687ms
15:	learn: 66296.2096525	total: 18.6ms	remaining: 677ms
16:	learn: 66068.4809624	total: 19.5ms	remaining: 668ms
17:	learn: 65857.3361333	total: 20.3ms	remaining: 656ms
18:

## Задание 8 (0.5 балла)

Теперь используете исключительно xgboost. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

In [ ]:
#ну я.. это сделала в 6м номере, но, видимо, стоит опять?

In [224]:
xg = XGBRegressor(learning_rate=0.01,
    booster = 'gblinear',              
    n_estimators=600,
    eval_metric=mean_absolute_error
)
start = time.time()
xg.fit (X_train, y_train)
end = time.time()
pred_val = xg.predict(X_valid)
pred_test = xg.predict(X_test)
en = time.time()
print('MAPE validation:', mean_absolute_percentage_error(y_valid, pred_val))
print('MAPE test:', mean_absolute_percentage_error(y_test, pred_test))
print('RMSE validation:', mean_squared_error(y_valid, pred_val, squared=False))
print('RMSE test:', mean_squared_error(y_test, pred_test, squared=False))
print('Время обучения: {0} мс, Время предсказания: {1} мс'.format(end-start, en - end))

MAPE validation: 0.774891137168269
MAPE test: 0.3669660098217635
RMSE validation: 42213.66506871121
RMSE test: 58779.70263118554
Время обучения: 0.26488757133483887 мс, Время предсказания: 0.007516622543334961 мс


## Задание 9 (0.5 балла)

Теперь используете исключительно lightgbm. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

In [225]:
from lightgbm import LGBMRegressor
lgb = LGBMRegressor(
    boosting_type='gbdt', 
    num_leaves=100, 
    max_depth=-1, 
    learning_rate=0.01, 
    n_estimators=600)
start = time.time()
lgb.fit(X_train, y_train)
end = time.time()
pred_val = lgb.predict(X_valid)
pred_test = lgb.predict(X_test)
en = time.time()

In [226]:
print('MAPE validation:', mean_absolute_percentage_error(y_valid, pred_val))
print('MAPE test:', mean_absolute_percentage_error(y_test, pred_test))
print('RMSE validation:', mean_squared_error(y_valid, pred_val, squared=False))
print('RMSE test:', mean_squared_error(y_test, pred_test, squared=False))
print('Время обучения: {0}, Время предсказания: {1}'.format(end-start, en - end))

MAPE validation: 0.7122785106268176
MAPE test: 0.3898085921606456
RMSE validation: 42682.81383305676
RMSE test: 60193.34338590134
Время обучения: 0.1672501564025879, Время предсказания: 0.0


## Задание 10 (0.5 балла)

Сделайте выводы про модели бустинга, какая из моделей показала лучший результат по качеству, скорости обучения и скорости предсказания

In [227]:
#быстрее всех обучился lightgbm и бысьрее всех предсказал. 

In [ ]:
#медленнее всех неожиданно обучался и предсказывал catboost, gxb был на втором месте. 
#однако по качеству на тесте lgb проиграл двум другим моделям. Лучшим по качеству оказался как раз таки catboost. 
#В итоге при сравнительно небольшом проигрыше в скорости (скорее всего, там можно ещё поподбирать гиперпараметры), catboost оптимальнее всего